In [1]:
import collections
import copy
import datetime
import itertools

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay

In [3]:
G.register_tournaments()

In [4]:
G.register_matchups()

In [6]:
data = [
  [("cibbl", "41656", 0, 855228, 856203),
  "856203",
  "11463900",
  [1, 1, "eaten", None],
  [1, 1, "eaten", "11463872"]],
  [("cibbl", "41741", 5, 855753, 856203),
  "855753",
  "STAR-Helmut Wulf",
  [1, 8, "secretWeaponBan", None],
  [1, 1, "blocked", "11463893"]],
  [("cibbl", "42040", 1, 856697, 859097),
  "859097",
  "11512475",
  [2, 4, "crowdPushed", None],
  [2, 4, "crowdPushed", "11472250"]],
  [("cibbl", "42425", 2, 864588, 867030),
  "864588",
  "11595390",
  [2, 6, "eaten", None],
  [2, 6, "eaten", "11595377"]],
 [("cibbl", "42425", 4, 864209, 866253),
  "866253",
  "11621107",
  [2, 6, "chainsaw", None],
  [2, 6, "chainsaw", "11621107"]],
 [("cibbl", "42426", 4, 854500, 862033),
  "862033",
  "STAR-Max Spleenripper",
  [1, 7, "secretWeaponBan", None],
  [1, 6, "blocked", "11436776"]],
 [("cibbl", "42426", 5, 862033, 863441),
  "862033",
  "11558603",
  [2, 1, "crowdPushed", None],
  [2, 1, "crowdPushed", "11582134"]],
 [("cibbl", "42748", 3, 872475, 872524),
  "872524",
  "STAR-Helmut Wulf",
  [2, 0, "secretWeaponBan", None],
  [1, 2, "blocked", "11721710"]],
 [("cibbl", "42887", 1, 873361, 874302),
  "874302",
  "STAR-Ritter Von Baum",
  [1, 2, "dodgeFail", None],
  [1, 2, "dodgeFail", "11737260"]],
 [("cibbl", "42892", 2, 856203, 867030),
  "856203",
  "11463899",
  [1, 1, "eaten", None],
  [1, 1, "eaten", "11463872"]],
 [("cibbl", "43151", 4, 869156, 874602),
  "874602",
  "11759479",
  [1, 7, "secretWeaponBan", None],
  [1, 1, "blocked", "11667457"]],
 [("cibbl", "43154", 1, 867862, 873945),
  "873945",
  "11747835",
  [2, 4, "crowdPushed", None],
  [2, 4, "crowdPushed", "11763541"]],
 [("cibbl", "43154", 2, 871312, 873945),
  "873945",
  "STAR-Flint Churnblade",
  [2, 0, "secretWeaponBan", None],
  [1, 6, "blocked", "11700284"]],
 [("cibbl", "43564", 2, 862033, 874302),
  "862033",
  "11841355",
  [2, 3, "crowdPushed", None],
  [2, 3, "crowdPushed", "11863513"]],
 [("cibbl", "43566", 4, 855753, 871787),
  "855753",
  "11455996",
  [2, 8, "crowdPushed", None],
  [2, 8, "crowdPushed", "11709013"]],
 [("cibbl", "43992", 0, 888233, 889245),
  "888233",
  "11972766",
  [1, 8, "eaten", None],
  [1, 8, "eaten", "11972753"]],
 [("cibbl", "43992", 0, 889542, 892658),
  "892658",
  "12044226",
  [1, 8, "crowdPushed", None],
  [1, 8, "crowdPushed", "12036400"]],
 [("cibbl", "44062", 1, 854678, 855228),
  "855228",
  "11494706",
  [1, 7, "crowdPushed", None],
  [1, 7, "crowdPushed", "11948519"]],
 [("cibbl", "44066", 2, 874302, 876774),
  "874302",
  "11754122",
  [2, 7, "crowdPushed", None],
  [2, 7, "crowdPushed", "11791176"]],
 [("cibbl", "44232", 2, 895004, 895015),
  "895015",
  "12083016",
  [1, 2, "bomb", None],
  [1, 2, "bomb", "12082644"]],
 [("cibbl", "44381", 5, 888994, 892248),
  "888994",
  "11985447",
  [2, 5, "crowdPushed", None],
  [2, 5, "crowdPushed", "STAR-Hungry Joe"]],
 [("cibbl", "44389", 2, 895929, 896228),
  "895929",
  "12098249",
  [1, 8, "secretWeaponBan", None],
  [1, 6, "chainsaw", None]],
 [("cibbl", "44390", 4, 897291, 897376),
  "897291",
  "12120800",
  [2, 2, "crowdPushed", None],
  [2, 2, "crowdPushed", "12163493"]],
 [("cibbl", "44462", 2, 898349, 898507),
  "898349",
  "12137868",
  [1, 6, "crowdPushed", None],
  [1, 6, "crowdPushed", "12140161"]],
 [("cibbl", "44747", 4, 891260, 898507),
  "891260",
  "12053070",
  [2, 4, "crowdPushed", None],
  [2, 4, "crowdPushed", "12149573"]],
 [("cibbl", "44747", 5, 885728, 897379),
  "885728",
  "12253372",
  [1, 3, "crowdPushed", None],
  [1, 3, "crowdPushed", "12182659"]],
 [("cibbl", "44750", 7, 874966, 879131),
  "879131",
  "STAR-Boomer Eziasson",
  [1, 1, "bomb", None],
  [1, 1, "bomb", "STAR-Boomer Eziasson"]],
 [("cibbl", "44751", 6, 871787, 882382),
  "882382",
  "STAR-Hack Enslash",
  [1, 7, "secretWeaponBan", None],
  [1, 1, "fouled", "11709010"]],
 [("cibbl", "44910", 2, 904377, 905924),
  "905924",
  "12254412",
  [2, 7, "bomb", None],
  [2, 7, "bomb", "12254414"]],
 [("cibbl", "45175", 0, 907959, 910344),
  "907959",
  "12285943",
  [1, 4, "crowdPushed", None],
  [1, 4, "crowdPushed", "12323846"]],
 [("cibbl", "45192", 1, 874317, 883051),
  "874317",
  "12239696",
  [2, 8, "crowdPushed", None],
  [2, 8, "crowdPushed", "12139597"]],
 [("cibbl", "45218", 4, 900747, 900800),
  "900747",
  "12329751",
  [1, 8, "eaten", None],
  [1, 8, "eaten", "12173110"]],
 [("cibbl", "45233", 3, 898507, 906259),
  "906259",
  "12271324",
  [1, 8, "crowdPushed", None],
  [1, 8, "crowdPushed", "12140169"]],
 [("cibbl", "45237", 4, 888760, 908529),
  "908529",
  "12295363",
  [1, 6, "crowdPushed", None],
  [1, 6, "crowdPushed", "11982021"]],
 [("cibbl", "45240", 1, 874452, 910344),
  "910344",
  "12323838",
  [1, 5, "crowdPushed", None],
  [1, 5, "crowdPushed", "12319539"]],
 [("cibbl", "45568", 1, 874317, 874545),
  "874545",
  "11942173",
  [1, 8, "crowdPushed", None],
  [1, 8, "crowdPushed", "11754417"]],
 [("cibbl", "45568", 4, 874545, 886114),
  "886114",
  "12441507",
  [2, 8, "crowdPushed", None],
  [2, 8, "crowdPushed", "11906271"]],
 [("cibbl", "45570", 1, 897353, 898507),
  "897353",
  "12320632",
  [1, 4, "crowdPushed", None],
  [1, 4, "crowdPushed", "12140161"]],
 [("cibbl", "45572", 4, 874302, 897376),
  "897376",
  "12163493",
  [2, 8, "crowdPushed", None],
  [2, 8, "crowdPushed", "11967732"]],
 [("cibbl", "45881", 1, 913702, 916031),
  "913702",
  "12378135",
  [1, 5, "crowdPushed", None],
  [1, 5, "crowdPushed", "12413279"]],
 [("cibbl", "45881", 3, 906026, 916031),
  "906026",
  "12255963",
  [1, 1, "bomb", None],
  [1, 1, "bomb", "12255963"]],
 [("cibbl", "45885", 1, 896134, 904411),
  "904411",
  "12231956",
  [1, 6, "crowdPushed", None],
  [1, 6, "crowdPushed", "12101583"]],
 [("cibbl", "45885", 2, 859097, 904411),
  "904411",
  "12231982",
  [1, 8, "crowdPushed", None],
  [1, 8, "crowdPushed", "11978579"]],
 [("cibbl", "45885", 4, 879131, 896134),
  "896134",
  "12489639",
  [2, 3, "crowdPushed", None],
  [2, 3, "crowdPushed", "12493747"]],
 [("cibbl", "45885", 6, 855120, 889245),
  "889245",
  "12435553",
  [1, 7, "crowdPushed", None],
  [1, 7, "crowdPushed", "12346201"]],
 [("cibbl", "45887", 6, 873349, 874317),
  "873349",
  "12401409",
  [1, 3, "crowdPushed", None],
  [1, 3, "crowdPushed", "11754417"]],
 [("cibbl", "46301", 0, 923140, 926021),
  "923140",
  "12523363",
  [1, 8, "eaten", None],
  [1, 8, "eaten", "12523353"]],
 [("cibbl", "46392", 3, 854507, 896972),
  "854507",
  "12239764",
  [1, 5, "crowdPushed", None],
  [1, 5, "crowdPushed", "12230477"]],
 [("cibbl", "46393", 2, 905886, 924998),
  "924998",
  "12554722",
  [2, 1, "crowdPushed", None],
  [2, 1, "crowdPushed", "12253780"]],
 [("cibbl", "46394", 1, 905924, 926596),
  "926596",
  "STAR-Garbage Throttlesnot",
  [2, 0, "secretWeaponBan", None],
  [1, 3, "chainsaw", "STAR-Garbage Throttlesnot"]],
 [("cibbl", "46687", 5, 886114, 889232),
  "886114",
  "12592844",
  [1, 4, "crowdPushed", None],
  [1, 4, "crowdPushed", "12718451"]],
 [("cibbl", "46707", 2, 896134, 908411),
  "896134",
  "12561659",
  [2, 3, "crowdPushed", None],
  [2, 3, "crowdPushed", "STAR-Big Gregg"]],
 [("cibbl", "46707", 5, 896134, 905924),
  "905924",
  "12254401",
  [1, 3, "ballAndChain", None],
  [1, 3, "bomb", "12462493"]],
 [("cibbl", "46709", 1, 871312, 909759),
  "909759",
  "12577744",
  [2, 4, "crowdPushed", None],
  [2, 4, "crowdPushed", "12456867"]],
 [("cibbl", "46709", 3, 896564, 904377),
  "896564",
  "12367446",
  [2, 8, "crowdPushed", None],
  [2, 8, "crowdPushed", "12231317"]],
]

In [8]:
data[0]

[('cibbl', '41656', 0, 855228, 856203),
 '856203',
 '11463900',
 [1, 1, 'eaten', None],
 [1, 1, 'eaten', '11463872']]

In [10]:
for matchup_KEY, teamId, playerId, olddead, newdead in data:
    Mu = Matchup(*matchup_KEY)
    d = Mu.config["player"][teamId][playerId]
    assert d["dead"] == olddead
    d["dead"] = newdead